In [ ]:
import sys
print(sys.path)

In [1]:
import os
import argparse
import base64
import json
import cv2
from datetime import datetime
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
import math
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
import tensorflow as tf
#tf.python.control_flow_ops = tf

new_size_col = 200
new_size_row = 66

def cropImage(image):
    # Preprocessing image files
    shape = image.shape
    # note: numpy arrays are (row, col)!
    image = image[70:shape[0]-20, 10:shape[1]-10]
#     image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)    
    return image

def resizeImage(image):
    # Preprocessing image files
    shape = image.shape
    image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)   
    return image

def preprocess(image):
    
    
    # get shape and chop off 1/3 from the top
    image = cropImage(image)
    image = resizeImage(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    # note: numpy arrays are (row, col)!
    #image = image[shape[0]-70:shape[0]-25, 0:shape[1]]
    #image = cv2.resize(image, (200,66), interpolation=cv2.INTER_AREA)
    #image = cv2.resize(image, (64,64), interpolation=cv2.INTER_AREA)
    return image



sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None





class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error
        
        cal_throttle = self.Kp * self.error + self.Ki * self.integral
        throttle = min(cal_throttle, 1)
        
        return throttle


controller = SimplePIController(0.1, 0.002)
set_speed = 25
controller.set_desired(set_speed)



@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_pre = np.asarray(image)
    image_array = preprocess(image_pre)
    #image_array = np.asarray(image)
    transformed_image_array = image_array[None, :, :, :]
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = 1.0*float(model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
#     throttle = controller.update(float(speed))
    if float(speed) > 25:
        throttle = 0.1
    else:
#         steering_angle = steering_angle *1.25
        throttle = 0.5
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)
    
    # save frame
    if image_folder != '':
        timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
        image_filename = os.path.join(image_folder, timestamp)
        cv2.imwrite('{}.jpg'.format(image_filename), image_array)
#         image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Remote Driving')
#     parser.add_argument('model', type=str,
#     help='Path to model definition json. Model weights should be on the same path.')
#     args = parser.parse_args()
#     with open(args.model, 'r') as jfile:
#         model = model_from_json(jfile.read())
    with open('model3.json', 'r') as jfile:
        model = model_from_json(jfile.read())

    model.compile("adam", "mse")
    weights_file = 'model3.h5'
#     weights_file = './tmp/result3.14-0.04.hdf5'   
    model.load_weights(weights_file)

    image_folder=''
    if image_folder != '':
        print("Creating image folder at {}".format(image_folder))
        if not os.path.exists(image_folder):
            os.makedirs(image_folder)
        else:
            shutil.rmtree(image_folder)
            os.makedirs(image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")
    
    
    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Using TensorFlow backend.


NOT RECORDING THIS RUN ...
connect  084d5972b92a42aeba8c54fe11981850


(2018) wsgi starting up on http://0.0.0.0:4567
(2018) accepted ('127.0.0.1', 42765)


-0.27951696515083313 0.5
-0.27951696515083313 0.5
-0.27951696515083313 0.5
-0.28038549423217773 0.5
-0.28038549423217773 0.5
-0.28038549423217773 0.5
-0.27914220094680786 0.5
-0.28358137607574463 0.5
-0.28358137607574463 0.5
-0.28363001346588135 0.5
-0.2777917981147766 0.5
-0.2777917981147766 0.5
-0.25269418954849243 0.5
-0.25269418954849243 0.5
-0.25269418954849243 0.5
-0.27095746994018555 0.5
-0.2773757874965668 0.5
-0.2773757874965668 0.5
-0.2655821144580841 0.5
-0.2739950716495514 0.5
-0.2739950716495514 0.5
-0.24270440638065338 0.5
-0.24270440638065338 0.5
-0.24270440638065338 0.5
-0.21223892271518707 0.5
-0.21223892271518707 0.5
-0.21223892271518707 0.5
-0.20887035131454468 0.5
-0.22636115550994873 0.5
-0.22636115550994873 0.5
-0.21083292365074158 0.5
-0.21083292365074158 0.5
-0.21083292365074158 0.5
-0.159263476729393 0.5
-0.159263476729393 0.5
-0.159263476729393 0.5
-0.12140236794948578 0.5
-0.13506127893924713 0.5
-0.13506127893924713 0.5
-0.1317218840122223 0.5
-0.10339885950

-0.10711316764354706 0.1
-0.12413621693849564 0.1
-0.12413621693849564 0.1
-0.12413621693849564 0.1
-0.12691134214401245 0.1
-0.1074366420507431 0.1
-0.1074366420507431 0.1
-0.13962219655513763 0.1
-0.12688018381595612 0.1
-0.12688018381595612 0.1
-0.0754687562584877 0.5
-0.0761636272072792 0.5
-0.0761636272072792 0.5
-0.0654921904206276 0.5
-0.06019362434744835 0.5
-0.06019362434744835 0.5
-0.0481748953461647 0.1
-0.0481748953461647 0.1
-0.0481748953461647 0.1
-0.04183930158615112 0.1
-0.04183930158615112 0.1
-0.04183930158615112 0.1
0.05326109007000923 0.1
0.05326109007000923 0.1
0.05326109007000923 0.1
0.09786573797464371 0.1
0.09080301970243454 0.1
0.09080301970243454 0.1
0.11642222851514816 0.5
0.10553331673145294 0.5
0.10553331673145294 0.5
0.10402435809373856 0.5
0.08992476016283035 0.5
0.08992476016283035 0.5
0.034598641097545624 0.1
0.04038669168949127 0.1
0.04038669168949127 0.1
0.021696077659726143 0.1
0.021696077659726143 0.1
0.021696077659726143 0.1
-0.006784649100154638 0

...

0.06346406787633896 0.1
0.06346406787633896 0.1
0.0871356874704361 0.1
0.0871356874704361 0.1
0.0871356874704361 0.1
0.042299456894397736 0.1
0.042299456894397736 0.1
0.042299456894397736 0.1
-0.001410335418768227 0.1
-0.001410335418768227 0.1
-0.001410335418768227 0.1
-0.0006497375434264541 0.1
-0.01437522191554308 0.1
-0.01437522191554308 0.1
0.02653406746685505 0.1
0.02653406746685505 0.1
0.02653406746685505 0.1
-0.015981396660208702 0.5
0.0005770224379375577 0.5
0.0005770224379375577 0.5
-0.020718222483992577 0.5
-0.026505008339881897 0.5
-0.026505008339881897 0.5
-0.01966235414147377 0.1
-0.01966235414147377 0.1
-0.01966235414147377 0.1
0.005182531196624041 0.1
0.005182531196624041 0.1
0.005182531196624041 0.1
0.05192648246884346 0.1
0.05192648246884346 0.1
0.05192648246884346 0.1
0.009173652157187462 0.1
0.019901659339666367 0.1
0.019901659339666367 0.1
0.0857577994465828 0.5
0.0857577994465828 0.5
0.0857577994465828 0.5
0.08684102445840836 0.5
0.08878733962774277 0.5
0.088787339

-0.07316543161869049 0.5
-0.07316543161869049 0.5
-0.015256794169545174 0.5
-0.021212607622146606 0.5
-0.021212607622146606 0.5
-0.006942527834326029 0.1
-0.006942527834326029 0.1
-0.006942527834326029 0.1
-0.019225897267460823 0.1
-0.019225897267460823 0.1
-0.019225897267460823 0.1
-0.040292348712682724 0.1
-0.040292348712682724 0.1
-0.040292348712682724 0.1
-0.06509332358837128 0.1
-0.03218141570687294 0.1
-0.03218141570687294 0.1
-0.0344771146774292 0.1
-0.018355170264840126 0.1
-0.018355170264840126 0.1
-0.004660196136683226 0.5
-0.010332034900784492 0.5
-0.010332034900784492 0.5
-0.030021192505955696 0.5
-0.04639003053307533 0.5
-0.04639003053307533 0.5
-0.013415820896625519 0.5
-0.013415820896625519 0.1
-0.013415820896625519 0.1
-0.04619258642196655 0.1
-0.04619258642196655 0.1
-0.04619258642196655 0.1
-0.046137746423482895 0.1
-0.043182145804166794 0.1
-0.043182145804166794 0.1
-0.06048868969082832 0.1
-0.06048868969082832 0.1
-0.06048868969082832 0.1
-0.042558856308460236 0.1
-

-0.06838475912809372 0.5
-0.14297549426555634 0.1
-0.13606809079647064 0.1
-0.13606809079647064 0.1
-0.1582731455564499 0.1
-0.1582731455564499 0.1
-0.1582731455564499 0.1
-0.20768596231937408 0.1
-0.20768596231937408 0.1
-0.20768596231937408 0.1
-0.23854252696037292 0.1
-0.26779526472091675 0.1
-0.26779526472091675 0.1
-0.1833687424659729 0.1
-0.1833687424659729 0.1
-0.1833687424659729 0.1
-0.1401783525943756 0.5
-0.1401783525943756 0.5
-0.1401783525943756 0.5
-0.051454149186611176 0.5
-0.051454149186611176 0.5
-0.051454149186611176 0.5
-0.013207775540649891 0.1
-0.003309881081804633 0.1
-0.003309881081804633 0.1
0.018993981182575226 0.1
0.018993981182575226 0.1
0.018993981182575226 0.1
0.08292695134878159 0.1
0.06165428087115288 0.1
0.06165428087115288 0.1
0.07892964780330658 0.1
0.07892964780330658 0.1
0.07892964780330658 0.1
0.04296158254146576 0.5
0.046561289578676224 0.5
0.046561289578676224 0.5
0.03439849987626076 0.5
0.03439849987626076 0.5
0.03439849987626076 0.5
-0.0310550816

-0.03303064778447151 0.1
-0.027013728395104408 0.1
-0.06613044440746307 0.1
-0.06613044440746307 0.1


127.0.0.1 - - [07/Nov/2017 01:16:57] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 100.313088
wsgi exiting
(2018) wsgi exited, is_accepting=True
